In [1]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(chrome_options=options)
driver.get("https://www.wikiaves.com.br/midias.php?tm=s&t=s&s=11644&o=mp")

/home/renan/anaconda3/envs/birds_project/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [13]:
title = driver.find_element_by_class_name('wa-registros-titulo')
complete_name = title.text.strip('Sons de ')
popular_name = complete_name.split(' ')[0]
cientific_name = complete_name[complete_name.find('(') + 1: complete_name.find(')')] 

print(complete_name)
print(popular_name)
print(cientific_name)

canário-da-terra (Sicalis flaveola)
canário-da-terra
Sicalis flaveola


In [2]:
audios = driver.find_elements_by_tag_name('audio')
for audio_src in audios:
    print(audio_src.get_attribute('src'))

https://s3.amazonaws.com/media.wikiaves.com.br/recordings/221/122476_5ed1d7842d2f179a53091e450aa5dd2f.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/711/117795_94feb667e2aa2af8a4c3aad15f614b22.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/0/118_21cdca17b921697d026df643e4450de5.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/6161/1616738_9f7f0ac1b5b5b53a35961af5645fccfc.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/862/268692_0064696e3b436429f7e8948501ba3b19.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/964/469876_cebda2644515c055536db07ba1c9471d.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/221/122067_8482835e4d554a23caf1d3af7d018b95.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/6251/1526180_6fb60b074d912405cc22dcd7239b7ea4.mp3
https://s3.amazonaws.com/media.wikiaves.com.br/recordings/7551/1557991_e80207276cee4daee26ded773e0026f3.mp3
https://s3.amazonaws.com/media.wikiaves.com.b

In [17]:
import requests
import re

def get_filename_from_cd(cd):
    """
    Get filename from content-disposition
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]


url = 'https://s3.amazonaws.com/media.wikiaves.com.br/recordings/6161/1616738_9f7f0ac1b5b5b53a35961af5645fccfc.mp3'
r = requests.get(url, allow_redirects=True)
filename = get_filename_from_cd(r.headers.get('content-disposition'))
open('Teste.mp3', 'wb').write(r.content)


923304